# Check missings:  images, videos and annotations
version: 1

info: 


author: nuno costa

In [2]:
from annotate_v5 import *
import platform 
import numpy as np
import pandas as pd
from IPython.display import Image, display
import copy
import os
from shutil import copyfile
import matplotlib.pyplot as plt
from matplotlib.image import imread
from matplotlib.patches import Rectangle
import random

In [ ]:
dataset="TAO"

In [ ]:
#Define root dir dependent on OS
rdir_dsets='D:/external_datasets/' #WARNING: DATASETS ROOT is OK?
rdir='D:/external_datasets/'+ dataset
if str(platform.platform()).find('linux')>-1:
    dirdir_dsets=rdir_dsets.replace('D:/','/mnt/d/')
    rdir=rdir.replace('D:/','/mnt/d/')
print('OS: {}'.format(platform.platform()))
print('root datasets dir: {}'.format(rdir_dsets))
print('root dir: {}'.format(rdir))

In [ ]:
#jsonfile
injsonfile_rdir="TAO_DIR/annotations/train" #"split_mola_fix_equal/test"
molajson =  json.load(open(rdir+injsonfile_rdir+'.json'))
for k in molajson:
    print(k, len(molajson[k]))

# A. Verify using dataset toolkit

In [ ]:
#TAO
if (dataset=="TAO"):
    #https://github.com/TAO-Dataset/tao
    #https://github.com/TAO-Dataset/tao/blob/master/docs/download.md
    TAO_ROOT=rdir+"/TAO_DIR")
    !"path/violent_action/python" "tao/scripts/download/verify.py" $TAO_ROOT --split train

# B. Verify using MOLA annotation tools

## 1. Import
#### #NOTE: work with ids and index so you can use numpy for faster operations

In [ ]:
# datasets name and id
dset_l=[]
dset_l_id=[]
try:
    for d in molajson['datasets']:
        dset_l.append(d['name'])
        dset_l_id.append(d['id'])
except: #manually add for example for only COCO
    dset_l=['COCO']
    dset_l_id=[1]
print(dset_l, dset_l_id)

In [ ]:
# categories name and id
cat_l=[]
cat_l_id=[]
cat_l_dset=[]
for c in molajson['categories']:
    cat_l.append(c['name'])
    cat_l_id.append(c['id'])
    try:
        cat_l_dset.append(dset_l[c['dataset']-1]) # dset_l index is same as id-1
    except:
        cat_l_dset.append(dset_l[0])
#print(cat_l_id)

In [ ]:
# images filepath and id
img_l=[]
img_l_id=[]
for c in molajson['images']:
    img_l.append(c['file_name'])
    img_l_id.append(c['id'])

In [ ]:
# videos filepath and id
try:
    vid_l=[]
    vid_l_id=[]
    for c in molajson['videos']:
        vid_l.append(c['name'])
        vid_l_id.append(c['id'])
except:
    print("No video")

In [ ]:
# tracks 
try:
    track_l_id=[]
    track_l_vidid=[]
    for c in molajson['videos']:
        track_l_id.append(c['id'])
        track_l_vidid.append(c['video_id'])
except:
    print("No video")

In [ ]:
# annotations category_id, image_id, bbox, and dataset
ann_catid=[]
ann_imgid=[]
ann_bbox=[]
ann_dset=[]
ann_trackid=[]
for an in tqdm(molajson['annotations']):
    ann_catid.append(an['category_id'])
    ann_imgid.append(an['image_id'])
    ann_bbox.append(an['bbox'])
    try: ann_trackid.append(an['track_id']) 
    try:
        ann_dset.append(an['dataset'])
    except:
        ann_dset.append(dset_l_id[0])

## 2. CHECK IMAGES

In [ ]:
img_missing_l=[]
for img in img_l:
    src = os.path.join(rdir_dsets, img)
    if not os.path.exists(src): 
        img_missing_l.append(img)
print(rdir_dsets)
print(len(img_missing_l))

### 2.1 CHECK ANNOTATIONS (with missing images)

In [ ]:
# Create image dict {id: image}
images = {'%g' % x['id']: x for x in molajson['images']}

# annotations category_id
new_annotations=[]
for an in tqdm(molajson['annotations']):
    try:
        if an['iscrowd']: continue
    except:
        print('missing "iscrowd" key')
    img = images['%g' % an['image_id']]
    h, w, f = img['height'], img['width'], img['file_name']


    # The Labelbox bounding box format is [top left x, top left y, width, height]
    box = np.array(an['bbox'], dtype=np.float64)
    box[:2] += box[2:] / 2  # xy top-left corner to center
    box[[0, 2]] /= w  # normalize x
    box[[1, 3]] /= h  # normalize y

    if (box[2] > 0.) and (box[3] > 0.):  # if w > 0 and h > 0
        src = os.path.join(rdir_dsets, f)
        if os.path.exists(src): new_annotations.append(an)
        else: print(">> missing : ", src )

print(len(molajson['annotations']))
print(len(new_annotations))
print(len(molajson['annotations'])-len(new_annotations))

### 2.2 CHECK Classes|categories (annotations and images for each class w/ EXCEL report) 

In [ ]:
#checks #TODO: SORT alphabetically
checks_l=[]
checks_l_catid=[]
checks_l_catdset=[]
check_method="all_cats"
if check_method=="all_cats": #Do for all category names, even with equal 
    checks_l=cat_l
    checks_l_catid=[[id] for id in cat_l_id]
    checks_l_catdset=[[dset] for dset in cat_l_dset]
    

print(checks_l[0:5])
print(checks_l_catid[0:5])
print(checks_l_catdset[0:5])
print(len(checks_l))
print(len(checks_l_catid))
print(len(checks_l_catdset))

In [ ]:
# get annotations checks
ann_catid_np=np.array(ann_catid)
ann_imgid_np=np.array(ann_imgid)
ann_bbox_np=np.array(ann_bbox)
ann_dset_np=np.array(ann_dset)
checks_l_imgid=[]
checks_l_bbox=[]
checks_l_dset=[]
for catids in tqdm(checks_l_catid):
    l_imgid=[]
    l_bbox=[]
    l_dset=[]
    for catid in catids:
        ann_idx = np.where(ann_catid_np==catid)[0].tolist() #annotation index of ids
        l_imgid.append(ann_imgid_np[ann_idx].tolist())
        l_bbox.append(ann_bbox_np[ann_idx].tolist())
        l_dset.append(ann_dset_np[ann_idx].tolist())
    checks_l_imgid.append(l_imgid)
    checks_l_bbox.append(l_bbox)
    checks_l_dset.append(l_dset)

In [ ]:
#INIT VARS
method="save_images" #"save_images": to save new images and excel for manual inspection; "": already saved don't need to repeat and the excel are
datadir="checks/"+injsonfile_rdir+"/" #root folder to save check method . #WARNING checks/original json that was used to save images and excel
folder=check_method+'/' #folder to save images and exel 
showimage=False #show images
startidx=0 # start index of image to save from each dataset
imgnr=3 # total number of images to save from each dataset
imgstep='random' # step between images: int | 'random' - int steps between images; 'rand' gets random list
#paths
path=os.path.join(rdir,datadir,folder) #path to folder
assure_path_exists(path)
excelpath=path+check_method+"_report_v1.xlsx"#path+check_method+"_report.xlsx"#path to excel

In [ ]:
#METHODS
if method=="save_images": # save images and excel to folder for manual edit
    df=pd.DataFrame({'checks_l': checks_l,'checks_l_catid': checks_l_catid, 'checks_l_catdset': checks_l_catdset })
    df['annotations_missing'] = np.empty((len(df), 0)).tolist()
    df['images_missing'] = np.empty((len(df), 0)).tolist()
    #save image for each check
    for i, check in enumerate(tqdm(checks_l)): #run for each check category
        firstclass=check
        if isinstance(firstclass, list): firstclass=firstclass[0] #first class
        print('\n>> '+firstclass+'...') #class
        classpath=os.path.join(path, firstclass) # path to folder for images of  firstclass
        classpath=parse_path(classpath)+'/' #make it a folder
        assure_path_exists(classpath)
        df=save_imgs(df, rdir_dsets, classpath, i, dset_l, checks_l, checks_l_catid, checks_l_bbox, checks_l_dset,
              checks_l_imgid, img_l, img_l_id, startidx=startidx, imgnr=imgnr, imgstep=imgstep, showimage=showimage)    
    df.to_excel(excelpath, index=False)   

In [ ]:
#IMPORT EXCEL MANUAL EDIT #WARNING: CHECK EXCEL FIRST (#NOTE: donte use classes with missing annotations and images)
df=pd.read_excel(excelpath)
display(df)        

## 3. CHECK VIDEOS

In [ ]:
vid_missing_l=[]
for vid in vid_l:
    src = os.path.join(rdir_dsets, vid)
    if not os.path.exists(src): 
        vid_missing_l.append(vid)
print(rdir_dsets)
print(len(vid_missing_l))

### 3.1 CHECK ANNOTATIONS (with missing ivideos)

In [ ]:
# Create image dict {id: image}
videos = {'%g' % x['id']: x for x in molajson['videos']}
tracks = {'%g' % x['id']: x for x in molajson['tracks']}
# annotations category_id
new_annotations=[]
for an in tqdm(molajson['annotations']):
    try:
        if an['iscrowd']: continue
    except:
        print('missing "iscrowd" key')
    
    vid = videos['%g' % tracks['%g' % an['track_id']]]
    h, w, f = vid['height'], vid['width'], vid['name']


    # The Labelbox bounding box format is [top left x, top left y, width, height]
    box = np.array(an['bbox'], dtype=np.float64)
    box[:2] += box[2:] / 2  # xy top-left corner to center
    box[[0, 2]] /= w  # normalize x
    box[[1, 3]] /= h  # normalize y

    if (box[2] > 0.) and (box[3] > 0.):  # if w > 0 and h > 0
        src = os.path.join(rdir_dsets, f)
        if os.path.exists(src): new_annotations.append(an)
        else: print(">> missing : ", src )
            
print(len(molajson['annotations']))
print(len(new_annotations))
print(len(molajson['annotations'])-len(new_annotations))